<h1 align="center">Toán ứng dụng và thống kê</h1>
<h2 align="center">Đồ án Diagonalizable Matrix</h2>
<h3 align="center">Ngày 17/07/2024</h3>

# 1. Thông tin sinh viên

- Họ và tên: Nguyễn Gia Huy
- MSSV: 22127154
- Lớp: 22CLC06

# 2. Giải thuật

## 2.1 Định nghĩa

### 2.1.1 Trị riêng và vector riêng

Cho $A$ là ma trận vuông, vector $v$ và giá trị $\lambda$ khác 0. Nếu $Av = \lambda v$ thì $\lambda$ được gọi là trị riêng của $A$, $v$ được gọi là vector riêng tương ứng với trị riêng $\lambda$.

Khi đó, ta có thể viết lại phương trình trên dưới dạng: $(A - \lambda I)v = 0$. Điều này có nghĩa là $v$ là nghiệm của hệ phương trình $Ax = \lambda x$.

Để tìm trị riêng và vector riêng của ma trận $A$, ta giải phương trình đặc trưng: $\text{det}(A - \lambda I) = 0$.

Trong đó $P_A(\lambda) = \text{det}(A - \lambda I)$ được gọi là đa thức đặc trưng của ma trận $A$.

### 2.1.2 Thuật toán chéo hóa ma trận vuông $A \in \mathbb{M}_n$

**Bước 1:** Tìm đa thức đặc trưng $P_A(\lambda) = \text{det}(A - \lambda I)$ của ma trận $A$. Nếu $P_A(\lambda)$ có tổng các lũy thừa khác $n$ thì $A$ là ma trận $A$ không chéo hóa được, ngược lại, tiếp tục với bước 2.

**Bước 2:** Tìm trị riêng và vector riêng của ma trận $A$. Tìm tất cả các nghiệm của phương trình đa thức đặc trưng. Với mỗi trị riêng $\lambda_i$, tìm cơ sở và số chiều của không gian nghiệm phương trình $(A - \lambda_i I)v = 0$. Nếu mỗi trị riêng $\lambda_i$ có số chiều không gian nghiệm nhỏ hơn lũy thừa của nó trong đa thức đặc trưng thì $A$ không chéo hóa được, ngược lại, tiếp tục với bước 3.

**Bước 3:** Với các vector trong cơ sở không gian nghiệm của mỗi trị riêng $\lambda_i$, ta sẽ tạo ma trận $P$ sao cho cột của $P$ là các vector riêng tương ứng với các trị riêng $\lambda_i$. Khi đó, $P$ làm chéo hóa được ma trận $A$ và $P^{-1}AP = D$ với $D$ là ma trận đường chéo. $diag(\lambda_1, \lambda_2, ..., \lambda_n)$.


# 3. Cài đặt

Trong phần này, em sẽ trình bày 3 hướng cài đặt giải thuật chéo hóa ma trận vuông $A \in \mathbb{M}_n$.

- Hướng 1: Chỉ sử dụng `cmath` để hỗ trợ giải phương trình bậc 3. Toàn bộ những hàm cần thiết khác sẽ được viết bằng tay. (Cách này chỉ chạy được với ma trận có kích thước nhỏ hơn 4).
- Hướng 2: Sử dụng thư viện `numpy` và `sympy` để hỗ trợ giải từng bước của giải thuật.
- Hướng 3: Sử dụng thư viện `numpy` để giải nhanh toàn bộ giải thuật trong một vài dòng lệnh.

## 3.1 Cài đặt không sử dụng thư viện

### 3.1.1 Lớp `Matrix`

Lớp `Matrix` mô phỏng một ma trận 2 chiều và cung cấp một số phương thức hỗ trợ thao tác trên ma trận. Đoạn mã dưới đây chỉ bao gồm các phương thức vừa đủ để giải thuật chéo hóa hoạt động. Mã nguồn đầy đủ của lớp này có thể được tìm thấy trong file `matrix.py` do em viết tại [đây](https://github.com/HZeroxium/MTH00051/blob/main/matrix.py).


In [47]:
class Matrix:
    def __init__(self, data: list[list]) -> None:
        self.data = data
        self.rows = len(data)
        self.columns = len(data[0])

    def __str__(self) -> str:
        result = ""
        for i in range(self.rows):
            for j in range(self.columns):
                self.data[i][j] = round(self.data[i][j], 10)
            result += str(self.data[i]) + "\n"
        return result

    def __repr__(self) -> str:
        return f"Matrix({self.data})"

    def __getitem__(self, index: int) -> list:
        return self.data[index]

    def __setitem__(self, index: int, value: list):
        self.data[index] = value

    def __add__(self, other) -> "Matrix":
        if self.rows != other.rows or self.columns != other.columns:
            raise ValueError("Matrices must have the same dimensions")
        new_data = []
        for i in range(self.rows):
            new_data.append(
                [self.data[i][j] + other.data[i][j] for j in range(self.columns)]
            )
        return Matrix(new_data)

    def __sub__(self, other) -> "Matrix":
        if self.rows != other.rows or self.columns != other.columns:
            raise ValueError("Matrices must have the same dimensions")
        new_data = []
        for i in range(self.rows):
            new_data.append(
                [self.data[i][j] - other.data[i][j] for j in range(self.columns)]
            )
        return Matrix(new_data)

    def __mul__(self, other: "Matrix") -> "Matrix":
        new_data = []
        if type(other) == Matrix:
            if self.columns != other.rows:
                raise ValueError(
                    "The number of columns in the first matrix must be equal to the number of rows in the second matrix"
                )
            for i in range(self.rows):
                new_data.append(
                    [
                        sum(
                            [
                                self.data[i][k] * other.data[k][j]
                                for k in range(self.columns)
                            ]
                        )
                        for j in range(other.columns)
                    ]
                )
        elif type(other) == int or type(other) == float:
            new_data = [[x * other for x in row] for row in self.data]
        return Matrix(new_data)

    def getColumn(self: "Matrix", index: int) -> list:
        return [self.data[i][index] for i in range(self.rows)]

    def getRow(self: "Matrix", index: int) -> list:
        return self.data[index]

    def transpose(self) -> "Matrix":
        new_data = []
        for j in range(self.columns):
            new_data.append([self.data[i][j] for i in range(self.rows)])
        return Matrix(new_data)

    def getMinor(self: "Matrix", i: int, j: int) -> "Matrix":
        new_data = [
            [self.data[x][y] for y in range(self.columns) if y != j]
            for x in range(self.rows)
            if x != i
        ]
        return Matrix(new_data)

    def getCofactor(self: "Matrix", i: int, j: int) -> float:
        return self.getMinor(i, j).getDeterminant() * (-1) ** (i + j)

    def getDeterminant(self: "Matrix") -> float:
        if self.rows != self.columns:
            raise ValueError("The matrix must be square")
        if self.rows == 1:
            return self.data[0][0]
        if self.rows == 2:
            return self.data[0][0] * self.data[1][1] - self.data[0][1] * self.data[1][0]
        determinant = 0
        for i in range(self.rows):
            determinant += (
                self.data[0][i] * self.getMinor(0, i).getDeterminant() * (-1) ** i
            )
        return determinant

    def getInverse(self: "Matrix") -> "Matrix":
        determinant = self.getDeterminant()
        if determinant == 0:
            raise ValueError("The matrix is not invertible")
        new_data = [
            [self.getCofactor(i, j) / determinant for j in range(self.columns)]
            for i in range(self.rows)
        ]
        return Matrix(new_data)

    def addColumn(self: "Matrix", column: list) -> "Matrix":
        if len(column) != self.rows:
            raise ValueError("The column must have the same length as the matrix")
        new_data = [self.data[i] + [column[i]] for i in range(self.rows)]
        return Matrix(new_data)

    def separateColumn(self: "Matrix", index: int) -> tuple["Matrix", list]:
        column = self.getColumn(index)
        new_data = [
            self.data[i][:index] + self.data[i][index + 1 :] for i in range(self.rows)
        ]
        return Matrix(new_data), column

    def addRow(self: "Matrix", row: list) -> "Matrix":
        if len(row) != self.columns:
            raise ValueError("The row must have the same length as the matrix")
        new_data = self.data + [row]
        return Matrix(new_data)

    def removeRow(self: "Matrix", index: int) -> "Matrix":
        new_data = self.data[:index] + self.data[index + 1 :]
        return Matrix(new_data)

    def swapRows(self: "Matrix", i: int, j: int) -> None:
        self.data[i], self.data[j] = self.data[j], self.data[i]

    def swapColumns(self: "Matrix", i: int, j: int) -> None:
        for k in range(self.rows):
            self.data[k][i], self.data[k][j] = self.data[k][j], self.data[k][i]

    def attach_matrix_horizontal(self: "Matrix", other: "Matrix") -> "Matrix":
        if self.rows != other.rows:
            raise ValueError("Matrices must have the same number of rows")
        new_data = [self.data[i] + other.data[i] for i in range(self.rows)]
        return Matrix(new_data)

    def separate_matrix_horizontal(
        self: "Matrix", index: int
    ) -> tuple["Matrix", "Matrix"]:
        new_data1 = [self.data[i][:index] for i in range(self.rows)]
        new_data2 = [self.data[i][index:] for i in range(self.rows)]
        return Matrix(new_data1), Matrix(new_data2)

    def is_identity(self: "Matrix") -> bool:
        if self.rows != self.columns:
            return False
        for i in range(self.rows):
            for j in range(self.columns):
                if i == j:
                    if self.data[i][j] != 1.0:
                        return False
                else:
                    if self.data[i][j] != 0.0:
                        return False
        return True

    def Guass_Jordan(self: "Matrix") -> "Matrix":
        for i in range(self.rows):
            if self.data[i][i] == 0:
                for j in range(i + 1, self.rows):
                    if self.data[j][i] != 0:
                        self.swapRows(i, j)
                        break
            if self.data[i][i] == 0:
                continue
            self.data[i] = [x / self.data[i][i] for x in self.data[i]]
            for j in range(self.rows):
                if j != i:
                    self.data[j] = [
                        self.data[j][k] - self.data[j][i] * self.data[i][k]
                        for k in range(self.columns)
                    ]
        return self

    def getRank(self: "Matrix") -> int:
        rank = 0
        for i in range(self.rows):
            if all([x == 0 for x in self.data[i]]):
                break
            rank += 1
        return rank

    def getNullity(self: "Matrix") -> int:
        return self.columns - self.getRank()

    def format(self: "Matrix") -> "Matrix":
        for i in range(self.rows):
            for j in range(self.columns):
                if type(self.data[i][j]) == int:
                    self.data[i][j] = float(self.data[i][j])
                if abs(self.data[i][j]) < 1e-10:
                    self.data[i][j] = 0
                if self.data[i][j] == -0.0:
                    self.data[i][j] = 0.0
                self.data[i][j] = round(self.data[i][j], 2)
        return self

    def trace(self: "Matrix") -> float:
        if self.rows != self.columns:
            raise ValueError("The matrix must be square")
        return sum([self.data[i][i] for i in range(self.rows)])

    def is_square(self: "Matrix") -> bool:
        return self.rows == self.columns

### 3.1.2 Hàm `Gauss_elimination` và `back_substitution`

Hàm `Gauss_elimination` và `back_substitution` được sử dụng để giải hệ phương trình tuyến tính. Chi tiết thuật toán và cách cài đặt đã được trình bày trong [Đồ án 1 - Gauss](https://github.com/HZeroxium/MTH00051/tree/main/Project01). Sau đây là tóm tắt về 2 hàm này:

- `Gauss_elimination`: Chuyển ma trận về dạng ma trận tam giác trên với phần tử chéo bằng 0 hoặc 1.
- `back_substitution`: Giải hệ phương trình tuyến tính với ma trận đã được chuyển về dạng tam giác trên.


In [48]:
def Gauss_elimination(matrix: "Matrix") -> "Matrix":
    """
    Performs Gaussian elimination on the given matrix.

    Args:
        matrix (Matrix): The matrix to perform Gaussian elimination on.

    Returns:
        Matrix: The matrix after Gaussian elimination has been applied.
    """
    for i in range(matrix.rows):
        pivot = matrix.data[i][i]
        if pivot == 0:
            for j in range(i + 1, matrix.rows):
                if matrix.data[j][i] != 0:
                    matrix.swapRows(i, j)
                    pivot = matrix.data[i][i]
                    break
        if pivot == 0:
            continue
        matrix.data[i] = [x / pivot for x in matrix.data[i]]
        for j in range(i + 1, matrix.rows):
            matrix.data[j] = [
                matrix.data[j][k] - matrix.data[j][i] * matrix.data[i][k]
                for k in range(matrix.columns)
            ]
    return matrix


def back_substitution(matrix: Matrix) -> list:
    """
    Perform back substitution to solve a system of linear equations represented by a matrix.

    Args:
        matrix (Matrix): The matrix representing the system of linear equations.

    Returns:
        list: The solution(s) to the system of linear equations. If the system has a unique solution,
            a list containing the values of the variables is returned. If the system has infinitely
            many solutions, a list of solution vectors is returned.
    Raises:
        None
    """
    matrix_a, _ = matrix.separateColumn(matrix.columns - 1)
    rankA = matrix_a.getRank()
    rankA_ = matrix.getRank()

    if rankA < rankA_:
        # print(">>> Hệ phương trình vô nghiệm")
        return []

    n = matrix.columns - 1
    free_variables = []

    while matrix.rows < matrix.columns - 1:
        matrix = matrix.addRow([0] * matrix.columns)

    while matrix.rows > matrix.columns - 1:
        matrix = matrix.removeRow(matrix.rows - 1)

    # Currently, the matrix is ​​in the form of the above triangular matrix, let's convert it into a main diagonal matrix
    for i in range(matrix.rows):
        if matrix.data[i][i] == 1:
            for j in range(i):
                matrix.data[j] = [
                    matrix.data[j][k] - matrix.data[j][i] * matrix.data[i][k]
                    for k in range(matrix.columns)
                ]
        else:
            if i <= n:
                free_variables.append(i)

    rankA_ = matrix.getRank()
    # Unique solution
    if rankA == rankA_ and rankA == n:
        # print(">>> Hệ phương trình có nghiệm duy nhất")
        return [matrix[i][n] for i in range(n)]

    # Infinite solutions
    sol = []
    temp = [0] * n
    for i in range(n):
        if i < matrix.rows:
            temp[i] = matrix.data[i][n]
    sol.append(temp)

    # Consider columns that contain free hidden content
    for i in free_variables:
        temp = [0] * n
        temp[i] = 1
        for j in range(n):
            if j != i:
                temp[j] = -matrix.data[j][i]
        sol.append(temp)

    # print(">>> Hệ phương trình có vô số nghiệm")
    return sol

### 3.1.3 Hàm `inverse` và `get_identity_matrix`

Hàm `inverse` được sử dụng để tìm ma trận nghịch đảo của một ma trận vuông. Hàm này sử dụng phương pháp Gauss-Jordan để tìm ma trận nghịch đảo. Chi tiết thuật toán và cách cài đặt đã được trình bày trong [Đồ án 2 - Gauss-Jordan](https://github.com/HZeroxium/MTH00051/tree/main/Project02). Sau đây là tóm tắt về hàm này:

- `inverse`: Tìm ma trận nghịch đảo của một ma trận vuông.
- `get_identity_matrix`: Tạo ma trận đơn vị với kích thước cho trước.


In [49]:
def get_identity_matrix(n: int) -> Matrix:
    data = [[1 if i == j else 0 for j in range(n)] for i in range(n)]
    return Matrix(data)


def inverse(A: Matrix) -> Matrix:
    """
    Find the inverse of a square matrix using the Gauss-Jordan method.

    Args:
        A (Matrix): The matrix to find the inverse of.

    Returns:
        Matrix: The inverse of the input matrix A.
    """
    if A.rows != A.columns:
        raise ValueError("The input matrix must be square")

    n = A.rows
    I = get_identity_matrix(n)
    augmented_matrix = A.attach_matrix_horizontal(I)
    reduced_matrix = augmented_matrix.Guass_Jordan()
    _, inverse_matrix = reduced_matrix.separate_matrix_horizontal(n)

    if not _.is_identity():
        return None
    return inverse_matrix

### 3.1.4 Hàm tìm trị riêng `find_eigenvalues`

Hàm `find_eigenvalues` được sử dụng để tìm trị riêng của một ma trận vuông. Hàm này sử dụng phương pháp tìm nghiệm của phương trình đặc trưng. Bên dưới là mô tả một số hàm hỗ trợ trong hàm `find_eigenvalues`:

- `solve_2nd_degree_equation`: Tìm nghiệm của phương trình bậc 2.
- `solve_3rd_degree_equation`: Tìm nghiệm của phương trình bậc 3.
- `get_characteristic_polynomial`: Tính đa thức đặc trưng của ma trận.
- `find_eigenvalues`: Tìm trị riêng của ma trận.
- `format_solution`: Định dạng kết quả trả về (xóa các nghiệm trùng nhau, sắp xếp các nghiệm theo thứ tự tăng dần, làm tròn các nghiệm).

Dưới đây là mã nguồn cài đặt hàm `find_eigenvalues` và các hàm hỗ trợ dựa theo thuật toán đã mô tả bên trên


In [50]:
import cmath


def solve_2nd_degree_polynomial(a, b, c) -> list:
    if a == 0:
        raise ValueError(
            "Coefficient 'a' must not be zero for a 2nd degree polynomial."
        )

    discriminant = b**2 - 4 * a * c

    if discriminant > 0:
        roots = [
            (-b + discriminant**0.5) / (2 * a),
            (-b - discriminant**0.5) / (2 * a),
        ]
    elif discriminant == 0:
        roots = [-b / (2 * a)]
    else:
        roots = [
            (-b + discriminant**0.5 * 1j) / (2 * a),
            (-b - discriminant**0.5 * 1j) / (2 * a),
        ]

    # Chuyển đổi kết quả thành số thực nếu phần ảo bằng 0
    real_roots = [root.real if abs(root.imag) < 1e-10 else root for root in roots]
    return real_roots


def solve_3rd_degree_polynomial(a, b, c, d) -> list:
    if a == 0:
        raise ValueError(
            "Coefficient 'a' must not be zero for a 3rd degree polynomial."
        )

    # Chuyển đổi phương trình thành dạng: t^3 + pt + q = 0
    p = (3 * a * c - b**2) / (3 * a**2)
    q = (2 * b**3 - 9 * a * b * c + 27 * a**2 * d) / (27 * a**3)

    # Sử dụng công thức Cardano
    discriminant = ((q**2) / 4) + ((p**3) / 27)
    # Round to 0 if very close to 0
    if abs(discriminant) < 1e-10:
        discriminant = 0
    if discriminant > 0:
        u = (-q / 2 + cmath.sqrt(discriminant)) ** (1 / 3)
        v = (-q / 2 - cmath.sqrt(discriminant)) ** (1 / 3)
        roots = [u + v - (b / (3 * a))]
    elif discriminant == 0:
        u = 0
        if q > 0:
            u = -((q / 2) ** (1.0 / 3))
        else:
            u = (-q / 2) ** (1.0 / 3)
        roots = [2 * u - (b / (3 * a)), -u - (b / (3 * a))]
    else:
        rho = cmath.sqrt(-(p**3) / 27)
        theta = cmath.acos(-q / (2 * rho))
        u = 2 * rho ** (1 / 3)
        roots = [
            u * cmath.cos(theta / 3) - (b / (3 * a)),
            u * cmath.cos((theta + 2 * cmath.pi) / 3) - (b / (3 * a)),
            u * cmath.cos((theta + 4 * cmath.pi) / 3) - (b / (3 * a)),
        ]

    # Chuyển đổi kết quả thành số thực nếu phần ảo bằng 0
    real_roots = [root.real if abs(root.imag) < 1e-10 else root for root in roots]
    return real_roots


def get_characteristic_polynomial(matrix: Matrix) -> list:
    if not matrix.is_square():
        raise ValueError("Matrix must be square to have a characteristic polynomial.")

    n = matrix.rows
    if n == 1:
        return [matrix[0][0]]
    elif n == 2:
        return [1, -matrix.trace(), matrix.getDeterminant()]
    elif n == 3:
        matrix_square = matrix * matrix
        a = 1
        b = -matrix.trace()
        c = (matrix.trace() ** 2 - matrix_square.trace()) / 2
        d = -matrix.getDeterminant()
        return [a, b, c, d]
    else:
        raise ValueError(
            "Characteristic polynomial is not implemented for matrices larger than 3x3."
        )


def find_eigenvalues(matrix: Matrix) -> list:
    if not matrix.is_square():
        raise ValueError("Matrix must be square to have a characteristic polynomial.")

    n = matrix.rows
    if n == 1:
        return [matrix[0][0]]
    elif n == 2:
        return solve_2nd_degree_polynomial(1, -matrix.trace(), matrix.getDeterminant())
    elif n == 3:
        a, b, c, d = get_characteristic_polynomial(matrix)
        return solve_3rd_degree_polynomial(a, b, c, d)
    else:
        raise ValueError(
            "Characteristic polynomial is not implemented for matrices larger than 3x3."
        )


def format_solution(roots: list) -> list:
    # Remove duplicate roots
    roots = list(set(roots))
    # If roots include complex numbers, return
    if any(isinstance(root, complex) for root in roots):
        return roots
    # Sort roots in ascending order
    roots.sort()
    # If a value is very close to an integer, round it to the nearest integer
    roots = [round(root) if abs(root - round(root)) < 1e-10 else root for root in roots]
    return roots

### 3.1.5 Hàm tìm vector riêng `find_eigenvectors`

Hàm `find_eigenvectors` được sử dụng để tìm vector riêng của một ma trận vuông. Hàm này sử dụng phương pháp giải hệ phương trình $(A - \lambda I)v = 0$ để tìm vector riêng tương ứng với trị riêng đã tìm được. Bên dưới là mã nguồn cài đặt hàm `find_eigenvectors` dựa theo thuật toán đã mô tả bên trên.


In [51]:
def find_eigenvectors(matrix: Matrix, eigenvalues: list) -> list:
    eigenvectors: list[list] = []
    for eigenvalue in eigenvalues:
        temp: Matrix = matrix - get_identity_matrix(matrix.rows) * eigenvalue
        temp = temp.addColumn([0 for _ in range(temp.rows)])
        temp = Gauss_elimination(temp)
        solutions: list[list] = back_substitution(temp)
        if len(solutions) > 1:
            # Xóa các nghiệm không phải nghiệm tự do
            if solutions[0] == [0 for _ in range(len(solutions[0]))]:
                solutions.pop(0)
            for vector in solutions:
                eigenvectors.append(vector)
    return eigenvectors

### 3.1.6 Hàm chéo hóa ma trận `diagonalize_matrix`

Hàm `diagonalize_matrix` được sử dụng để chéo hóa một ma trận vuông. Hàm này sử dụng các hàm đã được cài đặt ở trên để tìm trị riêng, vector riêng và chéo hóa ma trận. Bên dưới là mã nguồn cài đặt hàm `diagonalize_matrix` dựa theo thuật toán đã mô tả bên trên.


In [52]:
def diagonalize_matrix(matrix: Matrix) -> tuple[Matrix, Matrix, Matrix, bool]:
    eigenvalues = format_solution(find_eigenvalues(matrix))
    # print("Trị riêng:", eigenvalues)
    eigenvectors = find_eigenvectors(matrix, eigenvalues)
    # print("Vector riêng:", eigenvectors)
    if len(eigenvectors) != matrix.rows:
        return None, None, None, False
    matrix_P = Matrix(eigenvectors).transpose()
    # print(matrix_P)
    matrix_D = inverse(matrix_P) * matrix * matrix_P
    return matrix_P, matrix_D, inverse(matrix_P), True

### 3.1.7 Hàm `main`

Hàm `main` được sử dụng để chạy chương trình. Hàm này sẽ tạo danh sách ma trận từ **Bài tập tuần 4** và chéo hóa từng ma trận trong danh sách đó. Bên dưới là mã nguồn cài đặt hàm `main`.


In [53]:
def main():
    matrices: list[Matrix] = [
        Matrix([[-1, 3], [-2, 4]]),
        Matrix([[5, 2], [9, 2]]),
        Matrix([[1, -1, -1], [1, 3, 1], [-3, 1, -1]]),
        Matrix([[5, -1, 1], [-1, 2, -2], [1, -2, 2]]),
        Matrix([[1, 3, 3], [-3, -5, -3], [3, 3, 1]]),
        Matrix([[4, 0, -1], [0, 3, 0], [1, 0, 2]]),
        Matrix([[3, 4, -4], [-2, -1, 2], [-2, 0, 1]]),
        Matrix([[0, 0, -2], [1, 2, 1], [1, 0, 3]]),
        Matrix([[1, 0, 0], [1, 2, 0], [-3, 5, 2]]),
        Matrix([[4, 0, 1], [-2, 1, 0], [-2, 0, 1]]),
    ]

    n = len(matrices)

    for i in range(n):
        print(f"*** Ma trận câu", chr(ord("a") + i), ":")
        print(matrices[i])
        matrix_P, matrix_D, matrix_P_inv, is_diagonalizable = diagonalize_matrix(
            matrices[i]
        )
        if not is_diagonalizable:
            print("Không thể chéo hóa ma trận.")
            print("=" * 50)
            continue
        print("--> Matrix P:")
        print(matrix_P)
        print("--> Matrix P^-1:")
        print(matrix_P_inv)
        print("--> Matrix D:")
        print(matrix_D)
        print("=" * 50)


if __name__ == "__main__":
    main()

*** Ma trận câu a :
[-1, 3]
[-2, 4]

--> Matrix P:
[1.5, 1.0]
[1, 1]

--> Matrix P^-1:
[2.0, -2.0]
[-2.0, 3.0]

--> Matrix D:
[1.0, -0.0]
[0.0, 2.0]

*** Ma trận câu b :
[5, 2]
[9, 2]

--> Matrix P:
[-0.3333333333, 0.6666666667]
[1, 1]

--> Matrix P^-1:
[-1.0, 0.6666666667]
[1.0, 0.3333333333]

--> Matrix D:
[-1.0, -0.0]
[0.0, 8.0]

*** Ma trận câu c :
[1, -1, -1]
[1, 3, 1]
[-3, 1, -1]

--> Matrix P:
[0.25, -1.0, -1.0]
[-0.25, -0.0, 1.0]
[1, 1, 1]

--> Matrix P^-1:
[0.8, 0.0, 0.8]
[-1.0, -1.0, 0.0]
[0.2, 1.0, 0.2]

--> Matrix D:
[-2.0, 0.0, 0.0]
[0.0, 2.0, 0.0]
[0.0, 0.0, 3.0]

*** Ma trận câu d :
[5, -1, 1]
[-1, 2, -2]
[1, -2, 2]

--> Matrix P:
[-0.0, -1.0, 2.0]
[1.0, -1.0, -1.0]
[1, 1, 1]

--> Matrix P^-1:
[0.0, 0.5, 0.5]
[-0.3333333333, -0.3333333333, 0.3333333333]
[0.3333333333, -0.1666666667, 0.1666666667]

--> Matrix D:
[0.0, 0.0, 0.0]
[0.0, 3.0, -0.0]
[0.0, 0.0, 6.0]

*** Ma trận câu e :
[1, 3, 3]
[-3, -5, -3]
[3, 3, 1]

--> Matrix P:
[-1.0, -1.0, 1.0]
[1, -0.0, -1.0]
[-0.0, 1, 

## 3.2 Cài đặt sử dụng thư viện hỗ trợ

Trong phần này, ta có thể sử dụng thư viện `sympy` và `numpy` để hỗ trợ tính toán các bước trong thuật toán chéo hóa ma trận, việc sử dụng các hàm của thư viện có thể giúp chéo hóa cho ma trận vuông `n x n` tổng quát (điều mà cách cài đặt không sử dụng thư viện chỉ có thể thực hiện khi $n \lt 3$). Dưới đây là mã nguồn minh họa:


In [54]:
import numpy as np  # type: ignore
import sympy as sp  # type: ignore

# Ma trận A ban đầu
matrices_np: list[np.ndarray] = [
    np.array([[-1, 3], [-2, 4]]),
    np.array([[5, 2], [9, 2]]),
    np.array([[1, -1, -1], [1, 3, 1], [-3, 1, -1]]),
    np.array([[5, -1, 1], [-1, 2, -2], [1, -2, 2]]),
    np.array([[1, 3, 3], [-3, -5, -3], [3, 3, 1]]),
    np.array([[4, 0, -1], [0, 3, 0], [1, 0, 2]]),
    np.array([[3, 4, -4], [-2, -1, 2], [-2, 0, 1]]),
    np.array([[0, 0, -2], [1, 2, 1], [1, 0, 3]]),
    np.array([[1, 0, 0], [1, 2, 0], [-3, 5, 2]]),
    np.array([[4, 0, 1], [-2, 1, 0], [-2, 0, 1]]),
]

length = len(matrices_np)

for i in range(length):
    print(
        "====================================================================================================="
    )
    print(f"*** Câu", chr(ord("a") + i), ":")
    A = matrices_np[i]

    # Kích thước của ma trận
    n = A.shape[0]

    # Tạo ma trận đơn vị
    I = np.eye(n)

    # Ký hiệu lambda
    lambda_sym = sp.symbols("λ")

    # Ma trận (A - I*lambda)
    A_lambda_I = A - I * lambda_sym
    print("---> Ma trận A - Iλ:")
    print(A_lambda_I)

    # Chuyển đổi ma trận (A - I*lambda) sang dạng SymPy Matrix
    A_lambda_I_sym = sp.Matrix(A_lambda_I)

    # Tìm định thức của ma trận (A - I*lambda)
    char_poly = A_lambda_I_sym.det()
    print("---> Đa thức đặc trưng của ma trận:")
    print(char_poly)

    # Giải đa thức đặc trưng để tìm các trị riêng (eigenvalues)
    eigenvalues = sp.solve(char_poly, lambda_sym)
    print("---> Các trị riêng (eigenvalues):")
    print(eigenvalues)

    # Tìm các eigenvectors tương ứng với mỗi trị riêng
    eigenvectors = []
    for val in eigenvalues:
        eig_matrix = A_lambda_I_sym.subs(lambda_sym, val)
        eig_vectors = eig_matrix.nullspace()
        eigenvectors.append(eig_vectors)

    print("---> Các vector riêng (eigenvectors):")
    for i, vectors in enumerate(eigenvectors):
        print(f"===> Trị riêng {eigenvalues[i]}: {vectors}")

    # Kiểm tra xem có thể chéo hóa ma trận không
    if len(eigenvectors) != n:
        print("!!! Không thể chéo hóa ma trận.")
        continue

    # Tạo ma trận P từ các eigenvectors
    P = sp.Matrix.hstack(*[vec[0] for vec in eigenvectors])

    # Ma trận đường chéo D từ các eigenvalues
    D = sp.diag(*eigenvalues)

    print("===> Ma trận P (các vector riêng):")
    print(P)
    print("===> Ma trận đường chéo D:")
    print(D)

    # Kiểm tra kết quả bằng cách tính ngược lại A = P * D * P^(-1)
    A_reconstructed = P * D * P.inv()
    print("===> Kiểm tra lại ma trận A:")
    print(A_reconstructed)

*** Câu a :
---> Ma trận A - Iλ:
[[-1.0*λ - 1 3]
 [-2 4 - 1.0*λ]]
---> Đa thức đặc trưng của ma trận:
1.0*λ**2 - 3.0*λ + 2
---> Các trị riêng (eigenvalues):
[1.00000000000000, 2.00000000000000]
---> Các vector riêng (eigenvectors):
===> Trị riêng 1.00000000000000: [Matrix([
[1.5],
[  1]])]
===> Trị riêng 2.00000000000000: [Matrix([
[1.0],
[  1]])]
===> Ma trận P (các vector riêng):
Matrix([[1.50000000000000, 1.00000000000000], [1, 1]])
===> Ma trận đường chéo D:
Matrix([[1.00000000000000, 0], [0, 2.00000000000000]])
===> Kiểm tra lại ma trận A:
Matrix([[-1.00000000000000, 3.00000000000000], [-2.00000000000000, 4.00000000000000]])
*** Câu b :
---> Ma trận A - Iλ:
[[5 - 1.0*λ 2]
 [9 2 - 1.0*λ]]
---> Đa thức đặc trưng của ma trận:
1.0*λ**2 - 7.0*λ - 8
---> Các trị riêng (eigenvalues):
[-1.00000000000000, 8.00000000000000]
---> Các vector riêng (eigenvectors):
===> Trị riêng -1.00000000000000: [Matrix([
[-0.333333333333333],
[                 1]])]
===> Trị riêng 8.00000000000000: [Matrix(

Bảng sau mô tả các hàm được sử dụng trong cài đặt sử dụng thư viện hỗ trợ:

| Hàm            | Mô tả                                     |
| -------------- | ----------------------------------------- |
| `np.shape`     | Trả về kích thước của ma trận             |
| `np.eye`       | Tạo ma trận đơn vị                        |
| `sp.symbols`   | Tạo biến ký hiệu                          |
| `sp.Matrix`    | Tạo ma trận từ mảng                       |
| `sp.det`       | Tính định thức của ma trận                |
| `sp.solve`     | Giải hệ phương trình                      |
| `sp.subs`      | Thay thế giá trị của biến trong biểu thức |
| `sp.nullspace` | Tìm không gian nghiệm của hệ phương trình |
| `sp.hstack`    | Nối ma trận theo chiều ngang              |
| `sp.diag`      | Tạo ma trận đường chéo                    |


## 3.3 Cài đặt sử dụng thư viện

Để cài đặt một ma trận chéo hóa, ta có thể sử dụng thư viện `numpy` của Python. Thư viện này cung cấp hàm `numpy.linalg.eig` để tìm trị riêng và vector riêng của một ma trận. Sau đó, ta có thể sử dụng trị riêng và vector riêng để chéo hóa ma trận. Dưới đây là mã nguồn minh họa cách chéo hóa một ma trận bằng thư viện `numpy`.


In [55]:
import numpy as np

matrices_np: list[np.ndarray] = [
    np.array([[-1, 3], [-2, 4]]),
    np.array([[5, 2], [9, 2]]),
    np.array([[1, -1, -1], [1, 3, 1], [-3, 1, -1]]),
    np.array([[5, -1, 1], [-1, 2, -2], [1, -2, 2]]),
    np.array([[1, 3, 3], [-3, -5, -3], [3, 3, 1]]),
    np.array([[4, 0, -1], [0, 3, 0], [1, 0, 2]]),
    np.array([[3, 4, -4], [-2, -1, 2], [-2, 0, 1]]),
    np.array([[0, 0, -2], [1, 2, 1], [1, 0, 3]]),
    np.array([[1, 0, 0], [1, 2, 0], [-3, 5, 2]]),
    np.array([[4, 0, 1], [-2, 1, 0], [-2, 0, 1]]),
]

n = len(matrices_np)


def diagonalize_matrix_np(
    matrix: np.ndarray,
) -> tuple[np.ndarray, np.ndarray, np.ndarray, bool]:
    eigenvalues, eigenvectors = np.linalg.eig(matrix)

    # Check if the matrix is diagonalizable or not
    if not np.all(np.isreal(eigenvalues)):
        return None, None, None, False
    if not np.allclose(
        matrix, eigenvectors @ np.diag(eigenvalues) @ np.linalg.inv(eigenvectors)
    ):
        return None, None, None, False
    D = np.diag(eigenvalues)
    P = eigenvectors
    P_inv = np.linalg.inv(P)
    return P, D, P_inv, True


for i in range(n):
    print(f"*** Ma trận câu", chr(ord("a") + i), ":")
    print(matrices_np[i])
    P, D, P_inv, is_diagonalizable = diagonalize_matrix_np(matrices_np[i])
    if not is_diagonalizable:
        print("Không thể chéo hóa ma trận.")
        print("=" * 50)
        continue
    print("--> Ma trận P:")
    print(P)
    print("--> Ma trận P^-1:")
    print(P_inv)
    print("--> Ma trận D:")
    print(D)
    print("=" * 50)

*** Ma trận câu a :
[[-1  3]
 [-2  4]]
--> Ma trận P:
[[-0.83205029 -0.70710678]
 [-0.5547002  -0.70710678]]
--> Ma trận P^-1:
[[-3.60555128  3.60555128]
 [ 2.82842712 -4.24264069]]
--> Ma trận D:
[[1. 0.]
 [0. 2.]]
*** Ma trận câu b :
[[5 2]
 [9 2]]
--> Ma trận P:
[[ 0.5547002  -0.31622777]
 [ 0.83205029  0.9486833 ]]
--> Ma trận P^-1:
[[ 1.20185043  0.40061681]
 [-1.05409255  0.70272837]]
--> Ma trận D:
[[ 8.  0.]
 [ 0. -1.]]
*** Ma trận câu c :
[[ 1 -1 -1]
 [ 1  3  1]
 [-3  1 -1]]
--> Ma trận P:
[[ 2.35702260e-01 -7.07106781e-01 -5.77350269e-01]
 [-2.35702260e-01  3.73921902e-16  5.77350269e-01]
 [ 9.42809042e-01  7.07106781e-01  5.77350269e-01]]
--> Ma trận P^-1:
[[ 8.48528137e-01 -7.37670138e-17  8.48528137e-01]
 [-1.41421356e+00 -1.41421356e+00 -3.47882984e-16]
 [ 3.46410162e-01  1.73205081e+00  3.46410162e-01]]
--> Ma trận D:
[[-2.  0.  0.]
 [ 0.  2.  0.]
 [ 0.  0.  3.]]
*** Ma trận câu d :
[[ 5 -1  1]
 [-1  2 -2]
 [ 1 -2  2]]
--> Ma trận P:
[[-8.16496581e-01  5.77350269e-01  7.

Bảng sau mô tả ngắn gọn các hàm của thư viện `numpy` được sử dụng trong mã nguồn bên trên:

| Hàm                | Mô tả                                         |
| ------------------ | --------------------------------------------- |
| `numpy.array`      | Tạo một mảng từ một danh sách                 |
| `numpy.linalg.eig` | Tìm trị riêng và vector riêng của một ma trận |
| `numpy.diag`       | Tạo ma trận đường chéo từ một mảng            |
| `numpy.linalg.inv` | Tìm ma trận nghịch đảo của một ma trận vuông  |

Dựa vào kết quả chạy mô phỏng các ví dụ trong **Bài tập tuần 4**, ta có thể thấy rằng cả hai cách cài đặt đều cho kết quả chéo hóa giống nhau, chỉ khác về cách định dạng kết quả trả về.


# 4. Ứng dụng

Chéo hóa ma trận, hay còn gọi là phân rã thành các ma trận chéo (diagonalization), có rất nhiều ứng dụng trong các lĩnh vực khác nhau của toán học và khoa học máy tính. Một trong những ứng dụng phổ biến, đơn giản và gần gũi nhất của việc chéo hóa là giải hệ phương trình tuyến tính. Khi ma trận được chéo hóa, việc giải hệ phương trình trở nên dễ dàng hơn nhiều so với khi ma trận chưa được chéo hóa.

Để giải hệ phương trình $AX = B$, ta có thể thực hiện các bước sau:

- **Bước 1.** Chéo hóa ma trận $A$ thành ma trận $D$. Khi đó, ta có $$A = PDP^{-1}$$.
- **Bước 2.** Hệ phương trình trở thành $$PDP^{-1}X = B$$.
- **Bước 3.** Đặt $Y = PX$, ta có $$DY = P^{-1}BPX$$ Vì $D$ là ma trận đường chéo, nên việc giải hệ phương trình này trở nên dễ dàng hơn nhiều so với việc giải hệ phương trình ban đầu.
- **Bước 4.** Tìm $Y$ từ hệ phương trình $$DY = P^{-1}BPX$$.
- **Bước 5.** Tìm $X$ từ $$Y = PX$$.

Như vậy, việc chéo hóa ma trận giúp giảm độ phức tạp của việc giải hệ phương trình tuyến tính, đồng thời giúp tăng hiệu suất tính toán.

Dưới đây là mã nguồn minh họa việc giải hệ phương trình tuyến tính bằng cách chéo hóa ma trận.


In [56]:
def solve_linear_system(A, B):
    P, D, P_inv, is_diagonalizable = diagonalize_matrix_np(A)

    if not is_diagonalizable:
        raise ValueError("The matrix A is not diagonalizable")

    # Chuyển ma trận B sang hệ tọa độ mới
    B_new = P_inv @ B

    # Giải hệ phương trình tuyến tính mới
    Y = np.linalg.solve(D, B_new)

    # Tìm nghiệm của hệ phương trình tuyến tính ban đầu
    X = P @ Y

    return X


A = np.array([[4, 1], [2, 3]])
B = np.array([1, 2])

X = solve_linear_system(A, B)
print("Solution X:", X)

Solution X: [0.1 0.6]


# 5. Tổng kết

Như vậy, qua đồ án này, em đã tìm hiểu về cách chéo hóa một ma trận vuông bằng cách tìm trị riêng và vector riêng của ma trận đó. Em đã cài đặt giải thuật chéo hóa một ma trận vuông bằng cả hai cách: không sử dụng thư viện và sử dụng thư viện `numpy`. Kết quả chạy mô phỏng cho thấy cả hai cách cài đặt đều cho kết quả chéo hóa giống nhau.

- Cài đặt không sử dụng thư viện giúp ta hiểu rõ hơn về cách hoạt động của giải thuật chéo hóa ma trận cũng như các định nghĩa về trị riêng, vector riêng, đa thức đặc trưng.
- Cài đặt sử dụng thư viện giúp ta tiết kiệm thời gian và công sức trong việc cài đặt các hàm tính toán phức tạp.

Đồng thời, em cũng đã thấy rằng việc chéo hóa ma trận giúp giảm độ phức tạp của việc giải hệ phương trình tuyến tính, đồng thời giúp tăng hiệu suất tính toán.

# 6. Tài liệu tham khảo

1. Slide bài giảng
2. [NumPy Documentation](https://numpy.org/doc/stable/)
3. [SymPy Documentation](https://docs.sympy.org/latest/index.html)
